# Класификатор используемой модели: Болталка или Продуктовый чат

## Загрузим необходимые библиотеки

In [1]:
import os
import random
import numpy as np
import pandas as pd

import re
from corus import load_rudrec
from collections import Counter, defaultdict

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report

import tensorflow as tf
from tensorflow.keras.callbacks import Callback

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

from tqdm.notebook import tqdm
tqdm.pandas()

## Определим необходимые функции

In [2]:
# https://habr.com/ru/articles/677512/

In [3]:
def tokenize_sentences(text_corp): # Функцию разбиения текстов корпуса на токены с разбивкой по предложениям
    token_corp = []
    for text in text_corp:
        text = re.sub(r'\s+', ' ', text, flags=re.M)
        for sent in re.split(r'(?<=[.!?…])\s+', text):
            sent = sent.replace('\n',' ')
            for word in sent.split():
                token = re.search(r'[а-яёА-ЯЁa-zA-Z]+', word, re.I)
                if token is None:
                    continue
                token_corp.append(token.group().lower())
            token_corp.append('EOS') # В конце каждого предложения добавляем фиктивный токен
    return token_corp

In [4]:
# Функция возвращает словарь с группированными биграммами: ключи – отдельные токены, 
# в значение – список слов которые следуют за ними в корпусе с собственной частотой 
def get_bigramms(token_list):
    bigramm_corp = []
    for i in range(len(token_list)-1):
        bigramm = token_list[i] + ' ' + token_list[i+1]
        bigramm_corp.append(bigramm) # Получим список биграмм
    
    unique_token_count = len(set(bigramm_corp)) # кол-во уникальных биграмм
    bigramm_proba = {} # Создаю словарик для результата: Ключ - биграмма. Значение - вероятность
     
    count_bigramm = Counter(bigramm_corp) # Создаю словарь для хранения частот биграмм
    count_token = Counter(token_list) # Создаю словарь для хранения частот токенов        
        
    # Создаю словарь с группированными биграммами: ключи – отдельные токены, 
    # в значение – список слов которые следуют за ними в корпусе 
    # с собственной частотой (  { 'отель': [('отличным', 4.116e-06), ('вобщем', 2.058e-06), …)
    grouped_bigramms = defaultdict(list)
    for bigramm in set(bigramm_corp):
        first_word, second_word = bigramm.split()
        proba = (count_bigramm[bigramm] + 1) / (count_token[first_word] + unique_token_count) # Формула Лапласа
        grouped_bigramms[first_word].append((second_word, proba))
    return grouped_bigramms

In [5]:
# Функция генерации текста
def generate_texts(token_label, grouped_bigramm, label, count_text, count_sent, count_word):

    # Создаём словарь для подсчёта биграмм "исключений"
    exceptions_bigramm = defaultdict(int)
    # Создаём список уникальных токенов для старта предложения
    unique_token = list(set(token_label))

    texts = []
    for it_text in tqdm(range(count_text)):  # Цикл с диапазоном кол-ва текстов
        text = ''
        unique_word = set()

        # Цикл с диапазоном кол-ва предложений в тексте
        for it_sent in range(count_sent):
            len_sent = count_word
            # Генерим случайное слово для начала предложения для обеспечения стохастического процесса генерации предложения
            start_word = random.choice(unique_token)
            # Записываем в строку с финальным предложением первое стартовое слово
            final_sent = start_word
            # Множество уникальных слов, которые уже сгенерились в предложение (чтобы геенерация не зацикливалась)
            unique_word.add(start_word)

            for step in range(count_word):
                next_word = None  # Создаём переменную для нового слова
                frequency = 0  # Переменная-счётчик для частоты каждого нового слова

                # Проходим циклом по словарю с ключом биграммы и значением её частоты
                for second_word, freq in grouped_bigramm[start_word]:
                    bigramm = start_word + ' ' + second_word
                    # Устанавливаем значение максимального повторения слова в одном тексте
                    if exceptions_bigramm[bigramm] > 3:
                        continue
                    if freq > frequency and second_word not in unique_word and second_word != 'EOS':
                        next_word = second_word
                        frequency = freq  # Если второе слово проходит условие запоминаем его
                if next_word is None:  # Если подходящего по условиям слова не найдено, перезаписываем стартовое слово и начинаем поиск заново
                    start_word = random.choice(unique_token)
                    final_sent += ', ' + start_word
                    unique_word.add(start_word)
                else:
                    # Если после цикла нашли подходящее слова (которое запомнили в цикле) - записываем его в предложение
                    exceptions_bigramm[start_word + ' ' + next_word] += 1
                    start_word = next_word
                    final_sent += ' ' + next_word
                    unique_word.add(start_word)
            final_sent += '. '
            text += final_sent
        texts.append(text)

    generation_text_df = pd.DataFrame(texts, columns=['text'])  # Формируем фрейм из списка
    generation_text_df['class'] = label
    return generation_text_df[['text', 'class']]

In [6]:
class My_Metrics(Callback):
    def on_test_end(self, epoch, logs=None):
        valid_prediction = model.predict(valid_dataset).logits.argmax(axis=1)
        print('\n', classification_report(y_valid.astype('int32'), valid_prediction))

In [7]:
def get_prediction(sentence, tokenizer, model):
    tokenized_sent = tokenizer(sentence, padding='max_length', truncation=True, return_tensors='tf')
    pred = model(tokenized_sent).logits.numpy()
    return pred, pred.argmax()

## Загрузка двнных общей и продуктовой болталки

#### Загрузка данных общей болталки

In [8]:
cc_data = pd.read_csv('common_talker_data_clean.csv')

In [9]:
cc_data.drop(['Unnamed: 0'], axis=1, inplace=True)
cc_data.shape

(10000, 3)

In [10]:
cc_data.head(4)

,question,answer,class
0,вопрос о тдв давно и хорошо отдыхаем лично вам...,хомячка,1
1,как парни относятся к цветным линзам? если у д...,меня вобще прикалывает эта тема,1
2,что делать сегодня нашёл 2 миллиона рублей?,если это счастье действительно на вас свалилос...,1
3,эбу в двенашке называется итэлма что за эбу?,эбу электронный блок управления двигателем авт...,1


In [11]:
cc_data['text'] = cc_data['question'] + " " + cc_data['answer']

In [12]:
cc_data.drop(columns=['question', 'answer'], axis=1, inplace=True)

In [13]:
cc_data = cc_data[['text', 'class']]

In [14]:
cc_data.head(4)

,text,class
0,вопрос о тдв давно и хорошо отдыхаем лично вам...,1
1,как парни относятся к цветным линзам? если у д...,1
2,что делать сегодня нашёл 2 миллиона рублей? е...,1
3,эбу в двенашке называется итэлма что за эбу? ...,1


Берем обьединенные вопросы и ответы и клас определяем как 1

#### Загрузка данных продуктовой болталки

In [15]:
pc_data = pd.read_csv('bads_products_data_clean.csv')

In [16]:
pc_data.drop(['Unnamed: 0'], axis=1, inplace=True)
pc_data.shape

(1675, 7)

In [17]:
pc_data.head(4)

,bad_name,link,Description,Regular_Price,Discounted_Price,text,class
0,"SuperMins, Витамин D3, жидкость, 10 мл",https://itab.pro/products/supermins-vitamin-d3...,Жидкий витамин D3 (2000 МЕ) от SuperMins - это...,670,463,"['supermins', 'витамин', 'd3', 'жидкость', '10...",0
1,"BioExpert, Биоактивный магниевый комплекс (5 ф...",https://itab.pro/products/bioexpert-bioaktivny...,Биоактивный магниевый комплекс (5 форм) от Bio...,2271,0,"['bioexpert', 'биоактивный', 'магниевый', 'ком...",0
2,"Wolfsport+iTAB, Вкусный B-комплекс (вишня), шо...",https://itab.pro/products/wolfsportitab-vkusny...,Смотреть видео\nВ-комплекс от Wolfsport и iTAB...,3515,0,"['wolfsportitab', 'вкусный', 'bкомплекс', 'виш...",0
3,"BioExpert, Растительный мелатонин, жидкость, 3...",https://itab.pro/products/bioexpert-rastitelny...,Смотреть видеообзор.\nМелатонин от BioExpert -...,1500,1190,"['bioexpert', 'растительный', 'мелатонина', 'ж...",0


Перезаписываем текст без токенизации

In [18]:
pc_data['text'] = pc_data['bad_name'] + " " + pc_data['Description']

И оставляем только text и class

In [19]:
pc_data.drop(columns=['bad_name', 'link', 'Description', 'Regular_Price', 'Discounted_Price'], axis=1, inplace=True)

In [20]:
pc_data.head(4)

,text,class
0,"SuperMins, Витамин D3, жидкость, 10 мл Жидкий ...",0
1,"BioExpert, Биоактивный магниевый комплекс (5 ф...",0
2,"Wolfsport+iTAB, Вкусный B-комплекс (вишня), шо...",0
3,"BioExpert, Растительный мелатонин, жидкость, 3...",0


Установили клас 0

#### Посмотрим на дисбаланс классов

In [21]:
cc_data.shape[0], pc_data.shape[0]

(10000, 1675)

Так как присутствует дисбаланс классов то увеличим датасет продуктов используя генерацию текста на основе существующего

#### Устранение дисбаланса классов

Посмотрим на далинну количество слов в документах

In [22]:
pc_tokens_count = pc_data['text'].progress_apply(lambda x: len(x.split()))

  0%|          | 0/1675 [00:00<?, ?it/s]

In [23]:
pc_tokens_count.describe()

count    1675.000000
mean      239.714030
std       168.344203
min        23.000000
25%       130.000000
50%       190.000000
75%       300.500000
max      1392.000000
Name: text, dtype: float64

Для генерации текста возьмем за основу 192 токена на документ.

In [24]:
pc_sentence_count = pc_data['text'].progress_apply(lambda x: len(x.split("\n")))

  0%|          | 0/1675 [00:00<?, ?it/s]

In [25]:
pc_sentence_count.describe()

count    1675.000000
mean       15.063881
std         7.955603
min         2.000000
25%         9.000000
50%        14.000000
75%        19.000000
max        59.000000
Name: text, dtype: float64

Для генерации возьмем 15 предложений на документ

Удалим из признака Description - "\n"

In [26]:
pc_data['text'] = pc_data['text'].progress_apply(lambda x: re.sub(r"\n", ' ', x))

  0%|          | 0/1675 [00:00<?, ?it/s]

Преобразуем корпус в список документов

In [27]:
pc_source_text = pc_data['text'].values.tolist()

Токенизируем полученный список

In [28]:
pc_tokenized_text = tokenize_sentences(pc_source_text)

In [29]:
pc_tokenized_text[:20]

['supermins',
 'витамин',
 'd',
 'жидкость',
 'мл',
 'жидкий',
 'витамин',
 'd',
 'ме',
 'от',
 'supermins',
 'это',
 'must',
 'для',
 'детей',
 'и',
 'взрослых',
 'EOS',
 'ме',
 'в']

Получим словарь с группированными биграмами на основе токенизированного корпуса

In [30]:
pc_bigramms = get_bigramms(pc_tokenized_text)

In [31]:
pc_bigramms['высокой']

[('эффективностью', 6.137840554247002e-05),
 ('частоты', 2.3016902078426257e-05),
 ('усвояемостью', 2.3016902078426257e-05),
 ('сорбционной', 1.5344601385617504e-05),
 ('степени', 5.3706104849661265e-05),
 ('биодостоступностью', 2.3016902078426257e-05),
 ('усвояемости', 2.3016902078426257e-05),
 ('антагонистической', 2.3016902078426257e-05),
 ('концентрации', 0.00015344601385617506),
 ('разглаживающей', 1.5344601385617504e-05),
 ('температуре', 1.5344601385617504e-05),
 ('энергией', 1.5344601385617504e-05),
 ('активностью', 9.206760831370503e-05),
 ('умственной', 1.5344601385617504e-05),
 ('нагрузкой', 1.5344601385617504e-05),
 ('плотности', 1.5344601385617504e-05),
 ('дозировке', 1.5344601385617504e-05),
 ('работоспособности', 2.3016902078426257e-05),
 ('адаптации', 1.5344601385617504e-05),
 ('биоактивностью', 2.3016902078426257e-05),
 ('метаболической', 1.5344601385617504e-05),
 ('дозировкой', 2.3016902078426257e-05),
 ('абсорбцией', 1.5344601385617504e-05),
 ('термической', 2.301690

Сформируем датасет из случайно сгенерированных документов на основе словаря

In [32]:
print(f'Количество генерируемых документов: {cc_data.shape[0] - pc_data.shape[0]}\nКоличество токенов на предложение: {192 // 15}')

Количество генерируемых документов: 8325
Количество токенов на предложение: 12


In [33]:
pc_data_gen = generate_texts(pc_tokenized_text, pc_bigramms, label=0, count_text=8325, count_sent=15, count_word=12)

  0%|          | 0/8325 [00:00<?, ?it/s]

In [34]:
pc_data_gen['text'][0]

'избытка жиров и относительной влажности воздуха не выше с врачом по применению взрослым. разрыва стенок сосудов способствует улучшению работы мозга от до еды запивая стаканом воды. сберегающий технологический процесс старения организма к компонентам продукта беременность кормление грудью перед применением. песчаный желчегонные свойства комплекс для детей месте при температуре c столовые ложки мл. болеутоляющего ранозаживляющего противоревматического мочегонного желчегонного средства на порцию витамин в форме холекальциферола мкг. побочных эффектов масло cbd масла кбд начинайте, объемом, гиалурановой кислоты мг противопоказания индивидуальная. ультрафиолетовому излучению морозу ветру вредным воздействием внешних загрязнений но только высококачественных натуральных ингредиентов. расширении вен диосмин рутин кверцетин феникулярин относящийся, позвоночнику помогает поддерживать здоровье сердца нормализует. агрессия, климаксе благодаря чему характеризуется структурной стабильностью позволя

Понятно что получилась относительная ерунда по смыслу, но для данной задачи это не важно

Обьединим два продуктовых датасета

In [35]:
pc_data_full = pd.concat([pc_data, pc_data_gen], ignore_index=True)

In [36]:
pc_data_full.shape

(10000, 2)

Семплируем случайным образом

In [37]:
pc_data_full = pc_data_full.sample(frac=1).reset_index(drop=True)

#### Обьединяем продуктовый датасет и датасет общей болталки в один

In [38]:
full_data = pd.concat([cc_data, pc_data_full], ignore_index=True)

In [39]:
full_data.shape

(20000, 2)

Перемешаем данные

In [40]:
full_data = full_data.sample(frac=1).reset_index(drop=True)

In [41]:
full_data

,text,class
0,у высокоразвитых инопланетян - есть националис...,1
1,кто знает как эта хрень работает как-то у друг...,1
2,у меня зрение -4 я ношу очки говорят что очки ...,1
3,возможен ли капитализм без дешевой рабочей сил...,1
4,как вам рыбка очень вкусно выглядит,1
...,...,...
19995,как вконтакте кинуть что-нибуть на стену друга...,1
19996,"травяными и эффективную, райское, противоглист...",0
19997,чем правели эти боги афина афродита дионие гев...,1
19998,"экстрактно комплекс vita, лимфостаз серьезное ...",0


In [42]:
full_data['class'].value_counts()

1    10000
0    10000
Name: class, dtype: int64

Сохраним датасет

In [43]:
full_data.to_csv('clasifier_ds.csv')

#### Разобьем датасет на обучающую и тестовую выборки

In [44]:
X_train, X_valid, y_train, y_valid = train_test_split(full_data['text'], full_data['class'], test_size=0.2, shuffle=True, stratify=full_data['class'], random_state=21)

### Загрузим мультиязыковую предобученную модель BERT

In [45]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Выполняем токенизацию датасета для текста

In [46]:
train_tokenized = tokenizer(X_train.to_list(), padding='max_length', truncation=True, return_tensors='tf')
valid_tokenized = tokenizer(X_valid.to_list(), padding='max_length', truncation=True, return_tensors='tf')

#### Выполняем OHE для классов

In [47]:
num_classes = 2
train_labels = tf.keras.utils.to_categorical(y_train.to_list(), num_classes)
valid_labels = tf.keras.utils.to_categorical(y_valid.to_list(), num_classes)

#### Создаем датасеты

In [48]:
train_features = {x: train_tokenized[x] for x in tokenizer.model_input_names}
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
train_dataset = train_dataset.shuffle(len(train_tokenized)).batch(8)

valid_features = {x: valid_tokenized[x] for x in tokenizer.model_input_names}
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_features, valid_labels))
valid_dataset = valid_dataset.shuffle(len(valid_tokenized)).batch(8)

#### Оставим для обучения только слой классификации

In [49]:
model.layers[0].trainable = False

#### Определим оптимизатор и функцию потерь и скомпилируем модель

In [50]:
model.compile(optimizer="Adam", loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 177,854,978
Trainable params: 1,538
Non-trainable params: 177,853,440
_________________________________________________________________


### Обучим модель

In [57]:
model.fit(train_dataset, validation_data=valid_dataset, epochs=3)

2000/2000 [==============================] - 362s 181ms/step - loss: 0.0484 - accuracy: 0.9861 - val_loss: 0.0198 - val_accuracy: 0.9960


#### Оценим качество модели

In [58]:
model.evaluate(valid_dataset, callbacks=[My_Metrics()])

500/500 [==============================] - 61s 122ms/step

               precision    recall  f1-score   support

           0       0.57      0.57      0.57      2000
           1       0.57      0.57      0.57      2000

    accuracy                           0.57      4000
   macro avg       0.57      0.57      0.57      4000
weighted avg       0.57      0.57      0.57      4000

500/500 [==============================] - 127s 253ms/step - loss: 0.0198 - accuracy: 0.9960


[0.01977398805320263, 0.9959999918937683]

#### Сохраняем модель и токенайзер

In [53]:
tokenizer.save_pretrained('classifier_tk')
model.save_pretrained('classifier_model')

#### Загружаем ранее сохраненные токенайзер и модель

In [54]:
tokenizer_l = AutoTokenizer.from_pretrained('classifier_tk')
model_l = TFAutoModelForSequenceClassification.from_pretrained('classifier_model')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some layers from the model checkpoint at classifier_model were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at classifier_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification f

#### Протестируем классификацию

In [55]:
sentences = ['Привет','Как дела','витамины', 'Я нашел 2 миллиона вчера, что мне делать','Биоактивный магниевый комплекс','Мелатонин', 'витаминный комплекс']

In [56]:
for sent in sentences:
    pred, label = get_prediction(sent, tokenizer_l, model_l)
    print(f'{sent}\n\tПредсказание: {pred}\n\tПредсказаный класс: {label}')

Привет
	Предсказание: [[-1.8611946  1.8603203]]
	Предсказаный класс: 1
Как дела
	Предсказание: [[-2.3803205  2.4190655]]
	Предсказаный класс: 1
витамины
	Предсказание: [[ 1.5154307 -1.4790545]]
	Предсказаный класс: 0
Я нашел 2 миллиона вчера, что мне делать
	Предсказание: [[-3.6493368  3.7352328]]
	Предсказаный класс: 1
Биоактивный магниевый комплекс
	Предсказание: [[ 0.9874413 -0.9530679]]
	Предсказаный класс: 0
Мелатонин
	Предсказание: [[-1.1008012  1.1232004]]
	Предсказаный класс: 1
витаминный комплекс
	Предсказание: [[ 0.74465954 -0.7238703 ]]
	Предсказаный класс: 0


Ну в целом не плохо, думал что после генрации будет хуже